# Predict Future Price

---

### Import Libraries and Dependencies

In [1]:
# Initial imports
import pandas as pd
from pathlib import Path
import datetime as dt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline


### Read CSV / API into Pandas DataFrame

In [2]:
# Loading data
file_path = Path("Resources/Yahoo Finance/All Yahoo Data.csv")
All_Yahoo_Data = pd.read_csv(file_path)
All_Yahoo_Data.head()

df_AAPL = All_Yahoo_Data.loc[All_Yahoo_Data['Ticker'] == 'AAPL']
df_AAPL.head()

,Date,High,Low,Open,Close,Volume,Adj Close,Ticker
0,2010-01-04,7.660714,7.585000,7.622500,7.643214,493729600.0,6.593423,AAPL
1,2010-01-05,7.699643,7.616071,7.664286,7.656429,601904800.0,6.604822,AAPL
2,2010-01-06,7.686786,7.526786,7.656429,7.534643,552160000.0,6.499765,AAPL
3,2010-01-07,7.571429,7.466071,7.562500,7.520714,477131200.0,6.487749,AAPL
4,2010-01-08,7.571429,7.466429,7.510714,7.570714,447610800.0,6.530882,AAPL


### Get data directly from Yahoo from Yahoo Finance API

### Preparing Data

In [3]:
# Define features set
df = df_AAPL.drop(["Open", "High", "Low", "Adj Close", "Ticker"], axis=1)
df.head()

,Date,Close,Volume
0,2010-01-04,7.643214,493729600.0
1,2010-01-05,7.656429,601904800.0
2,2010-01-06,7.534643,552160000.0
3,2010-01-07,7.520714,477131200.0
4,2010-01-08,7.570714,447610800.0


### Calculate trading signals

In [5]:
# Calculate daily return
df['Returns'] = df['Close'].pct_change()

# Create Target column
df['Target'] = np.where(df['Returns'] >= 0, 1.0, -1)

# Calculate 50 day simple movering average
df['MA 50'] = df['Close'].transform(lambda x: x.rolling(window = 50).mean())

# Calculate 100 day simple movering average
df['MA 100'] = df['Close'].transform(lambda x: x.rolling(window = 100).mean())

# Calculate simple moving average ratio to capture the relationship between the two moving averages
df['MA Ratio'] = df['MA 100'] / df['MA 50']

# Calculate more indicators
# ....

df.tail()

,Date,Close,Volume,Returns,Target,MA 50,MA 100,MA Ratio
2511,2019-12-24,71.067497,48478800.0,0.000951,1.0,65.16060,59.401700,0.911620
2512,2019-12-26,72.477501,93121200.0,0.019840,1.0,65.43355,59.643125,0.911507
2513,2019-12-27,72.449997,146266000.0,-0.000379,-1.0,65.71070,59.875125,0.911193
2514,2019-12-30,72.879997,144114400.0,0.005935,1.0,65.99190,60.106325,0.910814
2515,2019-12-31,73.412498,100805600.0,0.007307,1.0,66.27810,60.331875,0.910284


### Create Machine Learning Model

### Train Model

### Backtest/Tweak Model

### Make Predictions

### Evaluate & Plot